Pyspark date functions

we use this in pipelines for

filtering recent data,
incremental loads,
partitioning data,
reporting,
time-based metrics
CDC pipelines

In [0]:
from pyspark.sql.functions import *
from datetime import date

data = [
    (1, date(2024, 1, 10)),
    (2, date(2024, 3, 15)),
    (3, date(2025, 1, 5))
]

df = spark.createDataFrame(data , ["id", "order_date"])

df.show()
df.printSchema()

In [0]:
# use case -> incremental pipelines, audit columns data freshness

df.withColumn("today", current_date()).show()

In [0]:
# datediff() -> difference between dates
# use case -> days since order, SLA monitoring, retention metrics

df.withColumn(
    "old_days", datediff(current_date(), col("order_date"))
).show()

In [0]:
#extract year / month / day
# use case -> partitioning , reporting, time analysis

df.withColumns({
    "year" : year(col("order_date")),
    "month" : month(col("order_date")),
    "day" : dayofmonth(col("order_date"))
}).show()

In [0]:
# add_months () shift date

# use case -> subscription expiry, billing 

df.withColumn(
    "next_month", add_months("order_date", 1)
).show()

In [0]:
# date_add() / date_sub() -> add or substract days

# use case -> delivery windows, deadline calculations

df.withColumn("plus_7_days", date_add("order_date", 7))\
    .withColumn("minus_7_days", date_sub("order_date",7)).show()

In [0]:
# Example : orders after 2024

df.filter(col("order_date") > "2024-01-01").show()
# spark automatically converts string to date, used in incremental loads

In [0]:
# to_date, -> string -> date conversion
# Real often comes like: "2024-01-10"
# conver string -> date
from pyspark.sql.functions import to_date

data = [(1, "2024-01-10")]

df2 = spark.createDataFrame(data, ["id", "date_str"])
df2.withColumn(
    "order_date",
    to_date(col("date_str"), "yyyy-MM-dd")
).show()

# very important in ETL pipelines.

In [0]:
# date_format() -> format date for reporting

df.withColumn(
    "formatted_date",
    date_format("order_date", "yyyy/MM/dd")
).show()

In [0]:
# Real pipeline Pattern(imp)
# string date -> to_date -> filter -> extract year/month -> aggregate
# example
df.filter(year("order_date") == 2024).show()

current_date() -> today

datediff() -> days difference

year/month/day -> extract parts

add_months() -> shift month

date_add/sub -> add/substract days

to_date() -> string -> date

date_format() -> display format

In [0]:
# Add today's date column

df2.withColumn(
    "today" , current_date()
).show()

In [0]:
# calculate days difference

df2.withColumns({
    "to_date" : to_date(col("date_str"), "yyyy-MM-dd"),
    "to_timestamp" : to_timestamp(to_date(col("date_str")), "yyyy-MM-dd HH:mm:ss"),
    "to_timestamp2" : to_timestamp(to_date(col("date_str")), "yyyy-MM-dd HH:mm:ss.SSS"),
    "old_days" : datediff( current_date(), to_date(col("date_str")))
                                   
}).show()

In [0]:
# Extract Year + Month

df.withColumns({
    "month" : month(to_date(col("order_date"))),
    "year" : year(to_date(col("order_date"))),
    "day" : dayofmonth(to_date(col("order_date")))

}).show()

In [0]:
# Add 30 days to order_date

df.withColumn(
    "plus_30_days", date_add(to_date(col("order_date"), "yyyy-MM-dd"), 30)).show()

In [0]:
# 5️⃣ Filter orders from 2024 only

df.filter(year(col("order_date")) == 2024).show()
          

In [0]:
# UDF (User defined function)

from pyspark.sql.functions import *
from pyspark.sql.types import StringType

data = [
    (1, 500),
    (2, 150),
    (3, 800)
]

df = spark.createDataFrame(data, ["id", "amount"])
df.show()

In [0]:
# create pyton function
# business rule : amount > 300 then high else low

def price_category(amount):
    if amount >= 300:
        return "high"
    else:
        return "Low"

In [0]:
#convert it to spark udf
from pyspark.sql.functions import udf

price_udf = udf(price_category, StringType())

In [0]:
# apply udf

df.withColumn(
    "category", price_udf("amount")
).show()

🚨 Problem with UDF

When you use UDF:

Spark cannot optimize

No Catalyst optimization

No predicate pushdown

Slower execution

Python ↔ JVM conversion overhead

Performance drops.


prefer built-in Spark functions over UDF because UDF breaks Spark optimization.

In [0]:
# built-in functions
from pyspark.sql.functions import col
df.withColumn(
    "category",
    when( col("amount") >= 300 ,"High").otherwise("Low")
).show()

In [0]:
# pandas udf better than normal

# apache Arrow -> vectorized execution
# runs faster because process batch of rows

# example

from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import IntegerType
import pandas as pd

@pandas_udf(IntegerType())
def double_value(x: pd.Series) -> pd.Series:
    return x * 2

df.withColumn("double_amount", double_value("amount")).show()

#task

amount >= 500 → "Premium"
else → "Regular"

Then implement same logic using when().

